In [3]:
# Versión
V = 4

In [2]:
# Librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Carreras y Facultades General

In [4]:
# Lista de Carreras a Analizar
data = pd.read_excel('Datos/puestoEstudiante2022.xlsx', sheet_name = 'C51 Distribución PE programa')
carreras = data['Unnamed: 2'].dropna().to_numpy().tolist()
facultades = ['DISTRIBUCIÓN DE PUESTOS ESTUDIANTE POR PROGRAMA ACADÉMICO 2021-1', 'Administración', 'Arquitectura y Diseño', 
              'Artes y Humanidades', 'Ciencias', 'Ciencias Sociales', 'Educación', 'Ingeniería', 'Total', 
              'Contaduría Internacional', 'Periodismo', 'Geociencias', 'Microbiología', 'Lenguajes y Est. Soc.', 
             'Ingeniería Ambiental', 'Ingeniería Eléctrica', 'Ingeniería de Alimentos', 'Lenguas y Cultura']
for fac in facultades:
    carreras.remove(fac)
    
carreras

['Administración',
 'Arquitectura',
 'Diseño',
 'Arte',
 'Historia del Arte',
 'Literatura',
 'Música',
 'Biología',
 'Física',
 'Matemáticas',
 'Química',
 'Antropología',
 'Ciencia Política',
 'Filosofía',
 'Historia',
 'Psicología',
 'Derecho',
 'Economía',
 'Gobierno y Asuntos Públicos',
 'Ingeniería Biomédica',
 'Ingeniería Civil',
 'Ingeniería Electrónica',
 'Ingeniería Industrial',
 'Ingeniería Mecánica',
 'Ingeniería Química',
 'Ingeniería de Sistemas',
 'Medicina']

In [6]:
# Programas y Facultades vinculadas
dataProgramasFacultades = pd.read_excel('Datos/facultadesProgramas.xlsx')
dataProgramasFacultades.head()

,Programa,Facultad
0,Administración,Administración
1,Contaduría Internacional,Administración
2,Arquitectura,Arquitectura y Diseño
3,Diseño,Arquitectura y Diseño
4,Arte,Artes y Humanidades


In [7]:
# Datos de población anual según carrera
dfPoblacion = pd.read_excel('Datos/CarpetasDatos/AnioPorColumna/Poblacion.xlsx')
dfPoblacion = dfPoblacion.fillna(0)
dfPoblacion.head()

,Facultad / Programa,1997-2,1998-2,1999-1,1999-2,2000-1,2000-2,2001-1,2001-2,2002-1,...,2016-2,2017-1,2017-2,2018-1,2018-2,2019-1,2019-2,2020-1,2020-2,2021-1
0,Administración,441.0,436.0,439.0,439.0,442.0,468.0,481.0,522.0,527.0,...,997.0,985.0,989.0,989.0,1019.0,1020.0,1037.0,1058.0,1159.0,1202.0
1,Contaduría internacional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,60.0,85.0,88.0,100.0,104.0,107.0,110.0,97.0,93.0
2,Arquitectura,415.0,450.0,444.0,425.0,417.0,388.0,406.0,383.0,400.0,...,731.0,732.0,724.0,715.0,681.0,718.0,720.0,723.0,691.0,722.0
3,Diseño,223.0,273.0,347.0,345.0,407.0,353.0,356.0,401.0,426.0,...,730.0,761.0,751.0,761.0,765.0,781.0,793.0,795.0,812.0,833.0
4,Arte,0.0,0.0,233.0,217.0,197.0,206.0,194.0,190.0,195.0,...,290.0,298.0,291.0,280.0,267.0,230.0,241.0,221.0,203.0,204.0


# Guardar Datos

In [10]:
# Año a guardar datos
anio = 2019

In [11]:
params = ['Principal', 'Secundario', 'AsistentesGraduados', 'Graduados', 'TransferenciasExternas',
          'ProfesDoctoradoMaestria', 'QuieroEstudiar', 'ProfesSexo', 'Puestos', 'Target']

# AnioPorColumna
paramsFormat1 = ['AsistentesGraduados', 'Graduados', 'Poblacion', 'TransferenciasExternas'] 

# AnioPorHoja
paramsFormat2 = ['ProfesSexo', 'QuieroEstudiar', 'ProfesDoctoradoMaestria', 'Puestos', 'Target']

    
nRows = len(carreras)**2 - len(carreras) + 1
nCols = len(params)

dataMatrix = np.zeros((nRows, nCols))
dataMatrix = dataMatrix.tolist()
dataMatrix[0] = params

n = 1
for i in range(len(carreras)):
    for j in range(len(carreras)):
        if(i != j):
            dataMatrix[n][0] = carreras[i]
            dataMatrix[n][1] = carreras[j]
            n = n + 1
            
for iParam in range(2,len(params)):
    if(params[iParam] in paramsFormat1):
        data = pd.read_excel('Datos/CarpetasDatos/AnioPorColumna/'+params[iParam]+'.xlsx')
        data = data.fillna(0)
        for n in range(1,nRows):
            carrerai = dataMatrix[n][0]
            carreraj = dataMatrix[n][1]
            poblacioni1 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carrerai][str(anio)+'-1'])
            poblacioni4 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carrerai][str(anio-4)+'-1'])
            poblacionj1 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carreraj][str(anio)+'-1'])
            poblacionj4 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carreraj][str(anio-4)+'-1'])
            if(params[iParam] == 'AsistentesGraduados'):
                dataMatrix[n][iParam] = float(data.loc[data['Unidad contratante'] == carreraj][str(anio)+'-1'])/poblacionj1 - float(data.loc[data['Unidad contratante'] == carrerai][str(anio-1)+'-1'])/poblacioni1
            if(params[iParam] == 'Graduados'):
                dataMatrix[n][iParam] = float(data.loc[data['Facultad / Programa'] == carreraj][str(anio)+'-1'])/poblacionj4 - float(data.loc[data['Facultad / Programa'] == carrerai][str(anio-1)+'-1'])/poblacioni4
            if(params[iParam] == 'TransferenciasExternas'):
                dataMatrix[n][iParam] = float(data.loc[data['Facultad / Programa'] == carreraj][str(anio)+'-1'])/poblacionj1 - float(data.loc[data['Facultad / Programa'] == carrerai][str(anio-1)+'-1'])/poblacioni1
    elif(params[iParam] in paramsFormat2):
        if(params[iParam] != 'Target'):
            data = pd.read_excel('Datos/CarpetasDatos/AnioPorHoja/'+params[iParam]+'.xlsx', sheet_name=str(anio))
            data = data.fillna(0)
        else:
            data = pd.read_excel('Datos/CarpetasDatos/AnioPorHoja/'+params[iParam]+'.xlsx', sheet_name=str(anio))
            data = data.fillna(0)
        for n in range(1,nRows):
            carrerai = dataMatrix[n][0]
            carreraj = dataMatrix[n][1]
            poblacioni1 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carrerai][str(anio)+'-1'])
            poblacioni4 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carrerai][str(anio-4)+'-1'])
            poblacionj1 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carreraj][str(anio)+'-1'])
            poblacionj4 = float(dfPoblacion.loc[dfPoblacion['Facultad / Programa'] == carreraj][str(anio-4)+'-1'])
            if(params[iParam] == 'ProfesSexo'):
                dataMatrix[n][iParam] = float(data.loc[data['Facultad / Departamento'] == carreraj]['% Mujeres']) - float(data.loc[data['Facultad / Departamento'] == carrerai]['% Mujeres'])
            if(params[iParam] == 'QuieroEstudiar'):
                dataMatrix[n][iParam] = float(data.loc[data['Facultad / Programa'] == carreraj]['Activo'])/poblacionj1 - float(data.loc[data['Facultad / Programa'] == carrerai]['Activo'])/poblacioni1
            if(params[iParam] == 'ProfesDoctoradoMaestria'):
                dataMatrix[n][iParam] = float(data.loc[data['Facultad / Departamento'] == carreraj]['D'])/poblacionj1 + float(data.loc[data['Facultad / Departamento'] == carreraj]['M'])/poblacionj1 - float(data.loc[data['Facultad / Departamento'] == carrerai]['D'])/poblacioni1 - float(data.loc[data['Facultad / Departamento'] == carrerai]['M'])/poblacioni1
            if(params[iParam] == 'Puestos'):
                dataMatrix[n][iParam] = float(data.loc[data['Unnamed: 0'] == carrerai][carreraj])
            if(params[iParam] == 'Target'):
                dataMatrix[n][iParam] = float(data.loc[data['Unnamed: 0'] == carrerai][carreraj])/(poblacioni1 + poblacionj1)
        
df = pd.DataFrame(dataMatrix[1:], columns = dataMatrix[0])
df.to_excel(r'Datos_Limpios/datos'+str(anio)+'_V'+str(V)+'.xlsx', index = False)